# Implement and Test Fast LCT Algorithms

### _Dan T. Abell and Boaz Nash (RadiaSoft LLC)_

$\rule[2pt]{15mm}{0.33pt}\ \LaTeX\ \text{macros}\ \rule[2pt]{15mm}{0.33pt}$
$$
%% math text
\newcommand{\mhsp}{\mskip{1.5mu}}
\newcommand{\hmhsp}{\mskip{0.75mu}}
\newcommand{\nmhsp}{\mskip{-1.5mu}}
\newcommand{\nhmhsp}{\mskip{-0.75mu}}
\newcommand{\ud}{\mathop{}\!\mathrm{d}}% upright d for differential
\newcommand{\ue}{\mathrm{e}}% upright e for Euler number
\newcommand{\ui}{\mathrm{i}}% upright i for unit imaginary
\newcommand{\uj}{\mathrm{j}}% upright j for unit imaginary
\newcommand{\uk}{\mathrm{k}}% upright k for unit imaginary
\newcommand{\sl}{\,/\,}
%%
%% derivatives
\newcommand{\dd}[3][]{\ud^{#1}{#2}/\nmhsp\ud{#3}^{#1}}
\newcommand{\dt}[2][]{\ud^{#1}{#2}/\nmhsp\ud{t}^{#1}}
\newcommand{\Dd}[3][]{\frac{\ud^{#1}{#2}}{\ud{#3}^{#1}}}
\newcommand{\Dt}[2][]{\frac{\ud^{#1}{#2}}{\ud{t}^{#1}}}
\newcommand{\ptdd}[3][]{\partial^{#1}{#2}/\partial{#3}^{#1}}
\newcommand{\ptDd}[3][]{\frac{\partial^{#1}{#2}}{\partial{#3}^{#1}}}
%%
%% vector operators
\DeclareMathOperator{\grad}{\nabla\nmhsp\nmhsp}
\DeclareMathOperator{\divrg}{{\nabla\cdot}\nmhsp\nhmhsp}
\DeclareMathOperator{\curl}{{\nabla\times}\nmhsp\nhmhsp}
%%
%% vectors
%% -- using \boldsymbol
% \newcommand{\uV}[1]{\hat{\boldsymbol{#1}}}% unit vector
% \newcommand{\V}[1]{\boldsymbol{#1}}% vector
% \newcommand{\uVg}[1]{\hat{\boldsymbol{#1}}}% unit vector
% \newcommand{\Vg}[1]{\boldsymbol{#1}}% vector
%% -- using \vec
\newcommand{\uV}[1]{\hat{{#1}}}% unit vector
\newcommand{\V}[1]{\vec{#1}}% vector
\newcommand{\uVg}[1]{\hat{{#1}}}% unit vector
\newcommand{\Vg}[1]{\vec{#1}}% vector
%%
%% other symbols and operators
\newcommand{\tr}[1]{\tilde{#1}}
$$
$\rule[2pt]{58.3mm}{0.33pt}$

---
## Introduction & Theory

In this notebook, we $\dots$

The _Linear Canonical Transform_, or LCT, of a function $f(u)$ is defined by the rule
$$
  \mathcal{L}_M[f](v) = \ue^{-\ui\pi/4} \sqrt{\beta}\int_{-\infty}^\infty
      \!f(u)\, \ue^{-\ui\pi(\alpha v^2 - 2\beta uv + \gamma u^2)} \ud{u}.
$$
(KB Wolf says that leading factor “requires some care”.)
The relation between the symplectic $2\times2$ matrix $M$ and
the parameters $(\alpha, \beta, \gamma)$ is given by the relation
$$
  \begin{pmatrix} A & B \\ C & D \end{pmatrix} = 
  \begin{pmatrix}
      \gamma/\beta              &       1/\beta \\
      \alpha\gamma/\beta - \beta & \alpha/\beta
  \end{pmatrix}.
$$
One may represent many of the well-known integral transforms,
including the Fourier transform, as special cases of the general LCT.

The LCT obeys the _group property_
$$
  \mathcal{L}_{M_2} \circ \mathcal{L}_{M_1} = \mathcal{L}_{M_2 \cdot M_1}.
$$
As a consequence, one may compute a given LCT as a composition
of simpler LCTs.

The algorithms given by [Koç](#ref:Koc-2011-thesis), and
by [Koç, _et al._](#ref:Koc-2008-DCLCT), first decompose
the symplectic matrix defining a given LCT into a product of
simpler symplectic matrices.
Those simpler matrices define specific special cases of the LCT:
scaling, chirp multiplication, and Fourier transform.
The group property obeyed by LCTs allows one to write any LCT
as a corresponding composition of those simpler transforms.
Because chirp multiplication enlarges the _effective_ time-bandwidth
product, we must also, at some points in the computation, resample
the signal.

This notebook is organized as follows: 

The operation of _scaling_ corresponds to the LCT with matrix
$$
  M_m = \begin{pmatrix}
    m &  0  \\
    0 & 1/m
  \end{pmatrix}.
$$
Its acts on a function $f(u)$ according to the rule
$$
  \bigl[ \mathcal{M}_m f \bigr](u) = \frac{1}{\sqrt{m}} f\Bigl(\frac{u}{m}\Bigr),
$$
or, equivalently,
$$
  \bigl[ \mathcal{M}_m f \bigr](m\cdot u) = \frac{1}{\sqrt{m}} f(u).
$$
_NB:_ If $m<0$, then we must introduce a factor of $\pm\mathrm{i}$,
with present evidence suggesting use of the positive root.

Aside from an overall phase factor, the _Fourier transform_ operation corresponds to the LCT with matrix
$$
  F_\text{LC} = \begin{pmatrix}
    0 & 1  \\
   -1 & 0
  \end{pmatrix}.
$$
More specifically, the LC Fourier transform acts on a function $f(u)$ according to the rule
$$
  \bigl[ \mathcal{F}_\text{LC} f \bigr](v) = \ue^{-\ui\mhsp\pi/4} \int_{-\infty}^\infty \ue^{-\ui\mhsp 2\pi u v} f(u) \ud{u}.
$$

In numerical work, we approximate that integral as
$$
  \int_{-\infty}^\infty \ue^{-\ui\mhsp 2\pi u v} f(u) \ud{u}
  \approx
  \int_{-P/2}^{P/2} \ue^{-\ui\mhsp 2\pi u v} f(u) \ud{u}
  \approx
  \frac{P}{N}\sum\nolimits_j^N \ue^{-\ui\mhsp 2\pi j k/N} f\Bigl(j\frac{P}{N}\Bigr)
  .
$$

The operation of _chirp multiplication_ corresponds to the LCT with matrix
$$
  Q_q = \begin{pmatrix}
    1 & 0  \\
   -q & 1
  \end{pmatrix}.
$$
Its acts on a function $f(u)$ according to the rule
$$
  \bigl[ \mathcal{Q}_q f \bigr](u) = \mathrm{e}^{-\mathrm{i}\pi{q}u^2}f(u).
$$

The _Linear Canonical Transform_, or LCT, in two degrees of freedom
is governed by a $4\times4$ real symplectic matrix 
$$
  M = \begin{pmatrix} A & B \\ C & D \end{pmatrix}.
$$
in this context, the symplecticity of $M$ means that the $2\times2$
submatrices $A$, $B$, $C$, and $D$ must obey the relations
[[Koç](#ref:Koc-2011-thesis)]
$$
  \begin{align}
    \tr{A}C &= \tr{C}A, & \tr{B}D &= \tr{D}B, & \tr{A}D - \tr{C}B &= I \\
    A\tr{B} &= B\tr{A}, & C\tr{D} &= D\tr{C}, & A\tr{D} - B\tr{C} &= I,
  \end{align}
$$
where the tilde $\bigl(\tilde\square\bigr)$ denotes matrix transposition,
and $I$ denotes the $2\times2$ identity matrix. _N.B:_&#160;The fact that
the submatrices obey these particular relations tells us that the matrix $M$
acts on phase-space variables given in the order $(q_1, q_2, p_1, p_2)$
(see, for example, [Dragt, §3.3](#ref:Dragt-2019-LieMethNLDynam)).
This means that if one extracts the ray optical matrix $M$
with the phase-space variables given in some other order,
e.g.&#160;$(q_1, p_1, q_2, p_2)$, then one must—for the purposes of
the computation described here—make sure to permute appropriately the
matrix entries of $M$.

For a function $f(\V{u})$, the LCT governed by matrix $M$ is defined by
the rule (see, for example, [Koç](#ref:Koc-2011-thesis)
or [Healy, _et al._](#ref:Healy-2016-LCTs))
$$
  \mathcal{L}_M[f](\V{u}) = \frac{1}{\sqrt{\det \ui{B}}}
    \int_{-\infty}^\infty \int_{-\infty}^\infty \!f(\V{v})\,
    \exp\bigl[\ui\pi\bigl(   \tr{\V{u}}DB^{-1}\V{u}
                          - 2\tr{\V{v}}B^{-1}\V{u}
                          +  \tr{\V{v}}B^{-1}A\V{v}\bigr)\bigr].
$$
(As in the 1D case, the phase of that leading factor
“requires some care”. See [Healy, _et al._](#ref:Healy-2016-LCTs).)

Like the 1D LCT, the 2D LCT also obeys the _group property_
$$
  \mathcal{L}_{M_2} \circ \mathcal{L}_{M_1} = \mathcal{L}_{M_2 \cdot M_1}.
$$
As a consequence, one may, even in the 2D case, compute a given LCT as
a composition of simpler LCTs. And this is the mechanism behind the
construction of fast algorithms for 2D LCTs.

### References

<a id='references'></a>

Herewith a collection of references. Most concern the linear canonical transform (LCT), with an emphasis on computation—and fast computation in particular. The two we consulted most heavily are references by [Koç](#ref:Koc-2011-thesis) and
by [Koç, _et al._](#ref:Koc-2008-DCLCT), which describe the 1D algorithm implemented here. [Campos and Figueroa](#ref:Campos-2011-FastLCT) describe a _very_ different algorithm, which might prove useful for comparison.

1. T. Alieva and M.J. Bastiaans, “Properties of the linear canonical integral transformation”, _J. Opt. Soc. Amer. A_, 24(11):3658–3665, Nov. 2007. [doi: 10.1364/JOSAA.24.003658](https://doi.org/10.1364/JOSAA.24.003658).
<a id='ref:Alieva-2007-PropLCT'></a>
2. R.G. Campos and J. Figueroa, “A fast algorithm for the linear canonical transform”, _Signal Process._, 91(6):1444–1447, June 2011. [doi: 10.1016/j.sigpro.2010.07.007](https://doi.org/10.1016/j.sigpro.2010.07.007).
<a id='ref:Campos-2011-FastLCT'></a>
3. A.J. Dragt, _Lie methods for nonlinear dynamics with applications to accelerator physics_, Nov. 2019. Latest version available at [the author’s website](http://www.physics.umd.edu/dsat/).
<a id='ref:Dragt-2019-LieMethNLDyn'></a>
4. A.J. Dragt and S. Habib, “How Wigner functions transform under symplectic maps”, June 1998. [arXiv:quant-ph/9806056](http://arxiv.org/abs/quant-ph/9806056)
<a id='ref:Dragt-1998-HowWignerFnsXform'></a>
5. P. Gonnet, R. Pachón, and L.N. Trefethen, “Robust rational interpolation and least-squares”, _Electron. Trans. Numer. Anal._, 38:146–167, 2011.
<a id='ref:Gonnet-2011-RobustRatInterp'></a>
6. J.J. Healy and J.T. Sheridan, “Fast linear canonical transforms”, _J. Opt. Soc. Amer. A_, 27(1):21–30, Jan. 2010. [doi: 10.1364/JOSAA.27.000021](https://doi.org/10.1364/JOSAA.27.000021).
<a id='ref:Healy-2010-FastLCT'></a>
7. J.J. Healy, M.A. Kutay, H.M. Ozaktas, and J.T. Sheridan, editors, _Linear Canonical Transforms: Theory and Applications_, volume&#8239;98 of _Springer Series in Optical Sciences_ (Springer, New York), 2016.
<a id='ref:Healy-2016-LCTs'></a>
8. B.M. Hennelly and J.T. Sheridan, “Generalizing, optimizing, and inventing numerical algorithms for the fractional Fourier, Fresnel, and linear canonical transforms”, _J. Opt. Soc. Amer. A_, 22(5):917–927, May 2005a. [doi: 10.1364/JOSAA.22.000917](https://doi.org/10.1364/JOSAA.22.000917).
<a id='ref:Hennelly-2005-GenNumLCT'></a>
9. B.M. Hennelly and J.T. Sheridan, “Fast numerical algorithm for the linear canonical transform”, _J. Opt. Soc. Amer. A_, 22(5):928–937, 2005b. [doi: 10.1364/JOSAA.22.000928](https://doi.org/10.1364/JOSAA.22.000928).
<a id='ref:Hennelly-2005-FastLCT'></a>
10. A. Koç, _Fast Algorithms for Digital Computation of Linear Canonical Transforms_, PhD dissertation, Stanford University, Stanford, CA, Mar. 2011.
<a id='ref:Koc-2011-thesis'></a>
11. A. Koç, H.M. Ozaktas, C. Candan, and M.A. Kutay, “Digital computation of linear canonical transforms”, _IEEE Trans. Signal Process._, 56(6):2383–2394, May 2008. [doi: 10.1109/TSP.2007.912890](https://doi.org/10.1109/TSP.2007.912890).
<a id='ref:Koc-2008-DCLCT'></a>
12. A. Koç, H.M. Ozaktas, and L. Hesselink, “Fast and accurate computation of two-dimensional non-separable quadratic-phase integrals”, _J. Opt. Soc. Amer. A_, 27(6):1288–1302, June 2010. [doi: 10.1364/JOSAA.27.001288](https://doi.org/10.1364/JOSAA.27.001288).
<a id='ref:Koc-2010-FastCLCT'></a>
13. A. Koç, H.M. Ozaktas, and L. Hesselink, “Fast and accurate algorithm for the computation of complex linear canonical transforms”, _J. Opt. Soc. Amer. A_, 27(9):1896–1908, Sept. 2010. [doi: 10.1364/JOSAA.27.001896](https://doi.org/10.1364/JOSAA.27.001896).
<a id='ref:Koc-2010-Fast2DNSLCT'></a>
14. Y. Nakatsukasa, O. Sète, and L.N. Trefethen, “The AAA algorithm for rational approximation”, _SIAM J. Sci. Comput._, 40(3):A1494–A1522, 2018. [doi: 10.1137/16M1106122](https://doi.org/10.1137/16M1106122).
<a id='ref:Nakatsukasa-2011-AAARatApprox'></a>
15. Y. Nakatsukasa and L.N. Trefethen. “An algorithm for real and complex rational minimax approximation”, _SIAM J. Sci. Comput._, 2020. A preprint version is available at https://arxiv.org/abs/1908.06001.
<a id='ref:Nakatsukasa-2020-RCMinimax'></a>
16. F.S. Oktem and H.M. Ozaktas, “Exact relation between continuous and discrete linear canonical transforms”, _IEEE Signal Process. Lett._, 16(8):727–730, May 2009. [doi: 10.1109/LSP.2009.2023940](https://doi.org/10.1109/LSP.2009.2023940).
<a id='ref:Oktem-2009-ContDiscLCT'></a>
17. R. Pachón, P. Gonnet, and J. van Deun, “Fast and stable rational interpolations in roots of unity and Chebyshev points”, _SIAM J. Numer. Anal._, 50(3):1713–1734, 2012. [doi: 10.1137/100797291](https://doi.org/10.1137/100797291). One may obtain this paper in [preprint form](https://core.ac.uk/download/pdf/1568392.pdf).
<a id='ref:Pachon-2012-RatInterpRootsUnity'></a>
18. S.-C. Pei and S.-G. Huang, “Fast discrete linear canonical transform based on CM-CC-CM decomposition and FFT”, _IEEE Trans. Signal Process._, 64(4):855–866, Feb. 2016. [doi: 10.1109/tsp.2015.2491891](https://doi.org/10.1109/tsp.2015.2491891).
<a id='ref:Pei-2016-FastDLCT'></a>
19. A. Stern, “Sampling of linear canonical transformed signals”, _Signal Process._, 86(7):1421–1425, July 2006a. [doi: 10.1016/j.sigpro.2005.07.031](https://doi.org/10.1016/j.sigpro.2005.07.031).
<a id='ref:Stern-2006-SamplingLCT'></a>
20. A. Stern, “Why is the linear canonical transform so little known?”, in G. Cristóbal, B. Javidi, and S. Vallmitjana, editors, _5th International Workshop on Information Optics (WIO’06)_, volume&#8239;860 of _AIP Conference Proceedings_, pages 225–234, Woodbury, NY, 2006b (AIP Press). [doi: 10.1063/1.2361224](https://doi.org/10.1063/1.2361224).
<a id='ref:Stern-2006-WhyLC'></a>
21. A. Stern, “Sampling of compact signals in offset linear canonical transform domains”, _Signal, Image and Video Processing_, 1(4):359–367, July 2007. [doi: 10.1007/s11760-007-0029-0](https://doi.org/10.1007/s11760-007-0029-0).
<a id='ref:Stern-2007-Sampling'></a>
22. K.B. Wolf, _Integral Transforms in Science and Engineering_, volume&#8239;11 of _Mathematical Concepts and Methods in Science and Engineering_, Plenum Press, New York, 1979. Available [here](https://www.fis.unam.mx/~bwolf/integral.html), from Bernardo Wolf’s website.
<a id='ref:Wolf-1979-IntegralXformsSE'></a>
23. K.B. Wolf, “A top-down account of linear canonical transforms”, _SIGMA Symmetry Integrability Geom. Methods Appl._, 8:033, June 2012. [doi: 10.3842/SIGMA.2012.033](https://doi.org/10.3842/SIGMA.2012.033).
<a id='ref:Wolf-2012-TopDownLCT'></a>
24. X. Yang, Q. Tan, X. Wei, Y. Xiang, Y. Yan, and G. Jin, “Improved fast fractional-Fourier-transform algorithm”, _J. Opt. Soc. Amer. A_, 21(9):1677–1681, Sept. 2004. [doi: 10.1364/JOSAA.21.001677](https://doi.org/10.1364/JOSAA.21.001677).
<a id='ref:Yang-2004-FastFrFt'></a>
25. L. Zhao, J.T. Sheridan, and J.J. Healy, “Unitary algorithm for nonseparable linear canonical transforms applied to iterative phase retrieval”, _IEEE Signal Process. Lett._, 24(6):814–817, Mar. 2017. [doi: 10.1109/LSP.2017.2684829](https://doi.org/10.1109/LSP.2017.2684829).
<a id='ref:Zhao-2017-UnitaryNSLCT'></a>

---
## Preamble

Relation of the Greek alphabet to the Latin keys (on Mac OS):

```
a b g  d e  z h u  i k l  m n j  o p  r s  t y f  x c v  w
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
α β γ  δ ε  ζ η θ  ι κ λ  μ ν ξ  ο π  ρ σ  τ υ φ  χ ψ ω  ς
    Γ  Δ               Λ      Ξ    Π    Σ    Υ Φ    Ψ Ω
```

### Required imports

In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
# import plotly.graph_objects
# import seaborn as sb
# import pandas as pd

In [2]:
# import math as m
import numpy as np
import time as tm
# import copy
# import os
# import re
# from scipy import interpolate as scinterp
# from scipy import integrate as scinteg
# from scipy import optimize as sciopt
# from scipy import special as scisf
# import pickle

In [3]:
# import scipy.ndimage
# import scipy.ndimage as ip
# from srwlib import *
from libLCT import *

In [4]:
# plt.style.available
# plt.style.use('Solarize_Light2')
# plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12

### Notebook directory

Where are we?

In [5]:
nb_dir = os.getcwd() + '/'
nb_dir

'/home/vagrant/jupyter/rslight/LCT/'

### Mathematical and physical constants

Define some mathematical constants:

In [6]:
# pi
π = np.pi

# golden ratio
gr = (1 + np.sqrt(5)) / 2

# roots
rt2 = np.sqrt(2.)

# degree to radian, and radian to degree
degree = π / 180.
d2r = degree
r2d = 1 / degree

Define the SI prefixes, which are particularly useful for converting data magnitudes in plots:

In [7]:
# SI prefixes
si_Q  = 1.e30  # quetta
si_R  = 1.e27  # ronna
si_Y  = 1.e24  # yotta
si_Z  = 1.e21  # zetta
si_E  = 1.e18  # exa
si_P  = 1.e15  # peta
si_T  = 1.e12  # tera
si_G  = 1.e9   # giga
si_M  = 1.e6   # mega
si_k  = 1.e3   # kilo
si_h  = 1.e2   # hecto
si_da = 1.e1   # deka
si_1  = 1.e0   # one
si_d  = 1.e-1  # deci
si_c  = 1.e-2  # centi
si_m  = 1.e-3  # milli
si_u  = 1.e-6  # micro
si_n  = 1.e-9  # nano
si_p  = 1.e-12 # pico
si_f  = 1.e-15 # femto
si_a  = 1.e-18 # atto
si_z  = 1.e-21 # zepto
si_y  = 1.e-24 # yocto
si_r  = 1.e-27 # ronto
si_q  = 1.e-30 # quecto

And some physical constants:

In [8]:
# Recommended Values for the Physical Constants have been published by
# CODATA for the years 2018 2014 2010 2006 2002 1998 1986 1973 1969
# scipy.constants uses 2018

import scipy.constants as sc
# [ key for key in sc.physical_constants.keys() if "Boltzmann" in key ]

clight  = sc.physical_constants["speed of light in vacuum"][0]
k_B = sc.physical_constants["Boltzmann constant"][0]
# m_e_MeV = sc.physical_constants["electron mass energy equivalent in MeV"][0]
# m_p_MeV = sc.physical_constants["proton mass energy equivalent in MeV"][0]

# phoot (light-nano-second / m)
phoot = clight * si_n

# m_e_eV  = m_e_MeV * si_M
# m_e_GeV = m_e_MeV / si_k
# m_p_eV  = m_p_MeV * si_M
# m_p_GeV = m_p_MeV / si_k

### Function definitions (utilities)

Function to list directory contents:

In [9]:
def ls(path = None):
    """
    List directory contents, exluding dot-files and vim backups.
    """
    return [ f for f in np.sort(os.listdir(path)) if not (f[0] == '.' or f[-1] == '~') ]

Functions to place lines on plots:

In [10]:
def hline(yl, x01, clr, wd = 1):
    plt.plot(x01, [yl, yl], clr, lw = wd)

def vline(xl, y01, clr, wd = 1):
    plt.plot([xl, xl], y01, clr, lw = wd)

Function to grab both min and max of a list:

In [11]:
def minmax(arr):
    """
    Compute and return the min and max of a given array.
    """
    return np.min(arr), np.max(arr)

Function to print a matrix in an easy-to-read form:

In [12]:
def print_matrix(M, fmt = '13.9f'):
    """
    Print matrix M row by row.
    """
    r, c = np.shape(M)
    fmtstr = '{0:' + fmt + '}' 
    for j in range(r):
        #print('', end = '  ')
        for k in range(c - 1):
            print(fmtstr.format(M[j, k]), end = '  ')
        print(fmtstr.format(M[j, c - 1]))

In [14]:
def rotation_22(theta):
    """
    Return the 2x2 rotation matrix that rotates vectors in
    the plane counter-clockwise (CCW) by angle theta.
    """
    c, s = (np.cos(theta), np.sin(theta))
    return [[c, -s], [s, c]]

---
### Functions from `LCTlib.py`

Function to create a $2\times2$ rotation matrix:

In [13]:
?rotation_22

Signature: rotation_22(theta)
Docstring:
Return the 2x2 rotation matrix that rotates vectors in
the plane counter-clockwise (CCW) by angle theta.
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

Functions for permuting between $n\times{n}$-block and $2\times2$-block forms:

In [14]:
?permute_nn_22

Signature: permute_nn_22(arr)
Docstring:
Permute the entries of the given array according to
the transformation from phase-space variables
  (q1,...,qn, p1,...,pn)
to phase-space variables
  (q1,p1, ..., qn,pn).

Vectors transform according to the rule
    v' = P.v,
and matrices transform according to the rule
    M' = P.M.tr(P),
where P denotes the apprpriate permutation matrix,
and tr(P) denotes its transpose.
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

In [15]:
?permute_22_nn

Signature: permute_22_nn(arr)
Docstring:
Permute the entries of the given array according to
the transformation from phase-space variables
  (q1,p1, ..., qn,pn)
to phase-space variables
  (q1,...,qn, p1,...,pn).

Vectors transform according to the rule
    v' = P.v,
and matrices transform according to the rule
    M' = P.M.tr(P),
where P denotes the apprpriate permutation matrix,
and tr(P) denotes its transpose.
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

Functions to convert between parameter sets {_A_, _B_, _C_, _D_&#8239;} and {_α_, _β_, _γ_&#8239;}
<br>
**NB:** We still need to understand, and address, the cases $B\rightarrow0$ and $\beta\rightarrow0$.

In [16]:
?convert_params_3to4

Signature: convert_params_3to4(alpha, beta, gamma)
Docstring:
Given LCT parameters (α,β,γ), return the associated 2x2 ABCD matrix.

Caveats: Not all authors use the same convention for the parameters
(α,β,γ): some reverse the rôles of α and γ.
We follow the notation of [Koç [7]](#ref:Koc-2011-thesis)
and [Koç, et al. [8]](#ref:Koc-2008-DCLCT),
also [Healy [4], ch.10](#ref:Healy:2016:LCTs).

Restrictions: The parameter beta may not take the value zero.

Arguments:
α, β, γ -- a parameter triplet defining a 1D LCT

Return a symplectic 2x2 ABCD martrix.
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

In [17]:
?convert_params_4to3

Signature: convert_params_4to3(M_lct)
Docstring:
   Given a symplectic 2x2 ABCD matrix, return the associated parameter triplet (α,β,γ).

   Caveats: Not all authors use the same convention for the parameters
   (α,β,γ): some reverse the rôles of α and γ.
   We follow the notation of [Koç [7]](#ref:Koc-2011-thesis)
   and [Koç, et al. [8]](#ref:Koc-2008-DCLCT),
   also [Healy [4], ch.10](#ref:Healy:2016:LCTs).

   Restrictions: The (1,2) matrix entry may not take the value zero.

** DTA: We need to decide how best to handle b very near zero.

   Arguments:
   M_lct -- a symplectic 2x2 ABCD martrix that defines a 1D LCT

   Return the parameter triplet α, β, γ.
   
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

ABSC: LCT Abscissae

In [18]:
?lct_abscissae

Signature: lct_abscissae(nn, du, ishift=False)
Docstring:
Return abscissae for a signal of length nn with spacing du.

With the default setting `ishift = False`, the abscissae will
be either centered on the origin (if an odd number of points),
or one step off of that (if an even number of points).
If one sets `ishift = True`, then the computed array of abscissae
will be shifted, or “rolled”, to place the origin at the beginning.

Arguments:
nn -- length of signal for which to construct abscissae,
        i.e. number of samples
du -- sample spacing
ishift -- a boolean argument: if True, the array of abcissae will
            be rotated so as to place the origin in entry [0]
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

RSMP: Resample the Signal

In [19]:
?resample_signal

Signature: resample_signal(k, in_signal, debug=False)
Docstring:
   Resample the input signal, and return the resultant signal.

   This function takes an input signal
       U = [dt, [u0, u1, ..., u_{n-1}]]
   and resamples it by a factor of k, returning the new signal
       SMPL(k){U} = [dt', [u0, u1, ..., u_{n'-1}]],
   where n' is _roughly_ k * n. The “roughly” has to do with the
   fact that k need not be an integer.

   This function requires interpolating the data to a new sample
   interval: The initial range is Δt = n * dt, with sample points
   at the centers of n equal-size intervals, and the function is
   taken to have period Δt. The points for the resampled signal
   will occupy the _same_ range Δt.

** DTA: This function currently uses 1D _linear_ interpolation.
        We should upgrade this to use a more sophisticated
        interpolation scheme, such as those available in SciPy
        or that described by Pachón, et al.

   Arguments:
       k -- factor by which to

SCL: Scaling

In [20]:
?scale_signal

Signature: scale_signal(m, in_signal)
Docstring:
Scale the input signal, and return the result.

This function implements the LCT version of 1D Scaling (SCL),
with parameter m acting on a given input signal:
    SCL(m): LCT[m 0, 0 1/m]{f}(m.x) <- f(x) / sqrt(|m|).
The input data must have the form
    [dX, [f_0, ..., f_{N-1}]],
where dX denotes the sample interval of the incoming signal.

Arguments:
m -- scale factor
in_signal -- the signal to transform, [dX, signal_array],
             where dX denotes the incoming sample interval

Return the scaled signal.
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

LCFT: LC Fourier Transform

In [21]:
?lct_fourier

Signature: lct_fourier(in_signal)
Docstring:
   Fourier transform the input signal, and return the result.

   This function implements the LCT version of a 1D Fourier transform (FT),
       FT(): LCT[0 1, -1 0]{f}(y) <- e^{-ii φ} FT(f),
   using numpy’s FFT to do the heavy lifting. As indicated here, the LCT
   version differs by an overall phase.

** DTA: KB Wolf remarks that correctly identifying the phase is a delicate
        matter. In light of that, we need to verify the phase used here.

   Argument:
   in_signal -- the signal to transform, [dX, signal_array], where dX
                denotes the incoming sample interval, and we assume the
                signal array is assumed symmetric (in the DFT sense)
                about the origin

   Return the transformed signal in the form [dY, e^{-ii φ} FFT(signal)].
   
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

CM: Chirp Multiplication

In [22]:
?chirp_multiply

Signature: chirp_multiply(q, in_signal)
Docstring:
Transform the input signal by chirp multiplication with parameter q.

This function implements the LCT version of chirp multiplication (CM)
with parameter q acting on a given input signal:
    CM(q): LCT[1 0, q 1]{f}(x) <- e^{-ii π q x^2}f(x).
The input data must have the form
    [dX, [f_0, ..., f_{N-1}]],
where dX denotes the sample interval of the incoming signal.

Arguments:
q -- CM factor
in_signal -- the signal to transform, [dX, signal_array],
             where dX denotes the incoming sample interval

Return the transformed signal.
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

DCMP: LCT Decomposition

Here we define a function to compute the appropriate decomposition
and insert the necessary resamplings.

In [23]:
?lct_decomposition

Signature: lct_decomposition(M_lct)
Docstring:
   Given an LCT matrix, M_lct, return a decomposition into simpler matrices. 

   Any symplectic 2x2 ABCD matrix that defines a linear canonical transform
   (LCT) may be decomposed as a product of matrices that each correspond to
   simpler transforms for which fast [i.e., ~O(N log N)] algorithms exist.
   The transforms required here are scaling (SCL), chirp multiplication (CM),
   and the Fourier transform (FT). In addition, we must sometimes resample
   the data (SMPL) so as to maintain a time-bandwidh product sufficient to
   recover the original signal.

** DTA: Must we handle separately the case B = M_lct[1,2] = 0?
        What about the case |B| << 1?

   The decompositions used here comes from the work of Koç, et al.,
   in _IEEE Trans. Signal Proc._ 56(6):2383--2394, June 2008.

   Argument:
   M_lct -- symplectic 2x2 matrix that describes the desired LCT

   Return an array of size mx2, where m denotes the total number of
   ope

As a convenience for future testing, we here also define functions
that generate the relevant LCT matrices in one degree of freedom:

In [24]:
?lctmx_scale

Signature: lctmx_scale(m)
Docstring: <no docstring>
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

In [25]:
?lctmx_chirp_x

Signature: lctmx_chirp_x(q)
Docstring: <no docstring>
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

In [26]:
?lctmx_fourier

Signature: lctmx_fourier()
Docstring: <no docstring>
File:      ~/jupyter/rslight/LCT/libLCT.py
Type:      function

Here we pull together our work above, and define a function to compute the
general linear canonical ransform.

In [27]:
?apply_lct

Signature: apply_lct(M_lct, in_signal)
Docstring:
   Apply LCT[M_lct] to a given input signal, and return the result.

   Given a symplectic 2x2 ABCD matrix that defines an LCT, decompose
   the matrix into a sequence of simpler operations, so as to achieve
   an operation count of ~O(N log N).

   The algorithm implemented here is that given by Koç, et al.
   in IEEE Trans. Signal Proc. 56(6):2383--2394, June 2008.

** DTA: Consider implementing one or more of the other known
        fast LCT algorithms. Doing so can help with verifying
        correctness, as well as allow us to learn something
        about the relative performance of different algorithms.

   Arguments:
   M_lct -- symplectic 2x2 matrix that describes the desired LCT
   in_signal -- the signal to transform, [ dX, signal_array ], where
                dX denotes the sample interval of the given signal

   Return the transformed signal in the form [ dY, LCT[M_lct](signal)].
   
File:      ~/jupyter/rslight/LCT/libLCT

Here we extend the above computation of the 1D LCT to the case of
uncoupled (also called _separable_) linear canonical transforms in
two dimensions.

In [28]:
?apply_lct_2d_sep

Signature: apply_lct_2d_sep(MX_lct, MY_lct, in_signal)
Docstring:
Apply LCT[M_lct] to a given input signal, and return the result.
In this case, the 4x4 matrix M_lct is assumed separable and defined
by the pair of 2x2 matrices MX_lct and MY_lct. In addition, the
input signal is taken to have the form ( dX, dY, signal_array ),
where dX and dY denote the sample intervals along the rows and
columns respectively of the given 2D signal_array.

Given a pair of symplectic 2x2 ABCD matrices that define an
uncoupled LCT in two dimensions, apply LCT[MX_lct] to each row,
and then LCT[MY_lct] to each column.

The algorithm implemented here is that given by Koç, et al.
in IEEE Trans. Signal Proc. 56(6):2383--2394, June 2008.

Arguments:
MX_lct -- symplectic 2x2 matrix that describes the desired LCT
MY_lct -- symplectic 2x2 matrix that describes the desired LCT
in_signal -- the signal to transform, [ dX, dY, signal_array], where
             dX and dY denote the sample intervals of the given
       

---
### Define functions for computing the 2D LCT

In [ ]:
mx = rotation_22(360 / 3 * degree)
my = rotation_22(360 / 3 * degree)
mx, my

In [ ]:
mm = np.eye(4)
mm[0:2,0:2] = mx
mm[2:4,2:4] = my
mm

In [ ]:
permute_22_nn(mm)

In [ ]:
np.all(permute_nn_22(_) == mm)

In [ ]:
ns = 32
dX = 2 * np.pi / ns
xv = lct_abscissae(ns, dX, ishift = False)

In [ ]:
ns = 48
dY = 2 * np.pi / ns
yv = lct_abscissae(ns, dY, ishift = False)

In [ ]:
sig_xy = [ [ (1 + np.cos(x)) * (1 + np.cos(y)) for x in xv ] for y in yv ]
in_signal_2d = ( dX, dY, sig_xy )

In [ ]:
idx = 0
n_rows = 1
n_cols = 2
sz_h = 7
sz_v = sz_h / 1.2
plt.figure(figsize = (n_cols * sz_h, n_rows * sz_v))
shading = 'gouraud'
# shading = 'nearest'

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
ax.set_aspect(1.0)
pcm = ax.pcolormesh(xv, yv, np.real(sig_xy), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title('input signal (real part)')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
ax.set_aspect(1.0)
pcm = ax.pcolormesh(xv, yv, np.imag(sig_xy), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title('input signal (imaginary part)')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

plt.tight_layout()
plt.show()

In [ ]:
idx = 0
n_rows = 1
n_cols = 2
sz_h = 7
sz_v = sz_h / 1.2
plt.figure(figsize = (n_cols * sz_h, n_rows * sz_v))
# shading = 'gouraud'
shading = 'nearest'

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
# ax.set_aspect(1.0)
pcm = ax.pcolormesh(xv, yv, np.real(sig_xy), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title('input signal (real part)')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
# ax.set_aspect(1.0)
pcm = ax.pcolormesh(xv, yv, np.imag(sig_xy), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title('input signal (imaginary part)')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

plt.tight_layout()
plt.show()

In [ ]:
in_signal_2d[:2], np.shape(in_signal_2d[-1])

In [ ]:
signal_2d_1 = apply_lct_2d_sep(mx, my, in_signal_2d)
signal_2d_2 = apply_lct_2d_sep(mx, my, signal_2d_1)
signal_2d_3 = apply_lct_2d_sep(mx, my, signal_2d_2)

In [ ]:
np.shape(signal_2d_1[-1]), np.shape(signal_2d_2[-1]), np.shape(signal_2d_3[-1])

In [ ]:
hx, hy = signal_2d_3[:2]
ny, nx = np.shape(signal_2d_3[-1])
nx * hx, ny * hy

In [ ]:
idx = 0
n_rows = 3
n_cols = 2
sz_h = 7
sz_v = sz_h / 1.2
plt.figure(figsize = (n_cols * sz_h, n_rows * sz_v))
# shading = 'gouraud'
shading = 'nearest'

hx, hy, sig_arr = signal_2d_1
ny, nx = np.shape(sig_arr)
local_xv = lct_abscissae(nx, hx)
local_yv = lct_abscissae(ny, hy)

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
pcm = ax.pcolormesh(local_xv, local_yv, np.real(sig_arr), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title(r'$R[120°]$(input signal), real part')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
pcm = ax.pcolormesh(local_xv, local_yv, np.imag(sig_arr), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title(r'$R[120°]$(input signal), imaginary part')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')


hx, hy, sig_arr = signal_2d_2
ny, nx = np.shape(sig_arr)
local_xv = lct_abscissae(nx, hx)
local_yv = lct_abscissae(ny, hy)

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
pcm = ax.pcolormesh(local_xv, local_yv, np.real(sig_arr), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title(r'$R[120°]^2$(input signal), real part')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
pcm = ax.pcolormesh(local_xv, local_yv, np.imag(sig_arr), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title(r'$R[120°]^2$(input signal), imaginary part')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')


hx, hy, sig_arr = signal_2d_3
ny, nx = np.shape(sig_arr)
local_xv = lct_abscissae(nx, hx)
local_yv = lct_abscissae(ny, hy)

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
# ax.set_aspect(1.0)
pcm = ax.pcolormesh(local_xv, local_yv, np.real(sig_arr), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title(r'$R[120°]^3$(input signal), real part')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')
plt.xlim(-np.pi, np.pi)
plt.ylim(-np.pi, np.pi)

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
# ax.set_aspect(1.0)
pcm = ax.pcolormesh(local_xv, local_yv, np.imag(sig_arr), cmap = plt.cm.viridis, shading = shading)
plt.colorbar(pcm)
plt.title(r'$R[120°]^3$(input signal), imaginary part')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')
plt.xlim(-np.pi, np.pi)
plt.ylim(-np.pi, np.pi)

plt.tight_layout()
plt.show()

In [ ]:
idx = 0
n_rows = 1
n_cols = 2
sz_h = 7
sz_v = sz_h / 1
plt.figure(figsize = (n_cols * sz_h, n_rows * sz_v))
# shading = 'gouraud'
shading = 'nearest'

sig_arr = signal_2d_1[-1]

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
ax.pcolormesh(xv, yv, np.real(signal_2d_1[-1]), cmap = plt.cm.viridis, shading = shading)
# plt.colorbar()
# ax.set_aspect(1.0)
plt.title('input signal (real part)')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

idx += 1
ax = plt.subplot(n_rows, n_cols, idx)
# ax.colorbar()
ax.pcolormesh(xv, yv, np.imag(signal_2d_1[-1]), cmap = plt.cm.viridis, shading = shading)
# ax.set_aspect(1.0)
plt.title('input signal (imaginary part)')
plt.xlabel(r'$x \,/\, \mathrm{m}$')
plt.ylabel(r'$y \,/\, \mathrm{m}$')

plt.tight_layout()
plt.show()

In [ ]:
signal_2d = apply_lct_2d_sep(mx, my, in_signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
#  5

In [ ]:
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
# 10

In [ ]:
np.shape(signal_2d[-1])

In [ ]:
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
# 15

In [ ]:
np.shape(signal_2d[-1])

In [ ]:
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
# 20

In [ ]:
np.shape(signal_2d[-1])

In [ ]:
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
# 25

In [ ]:
np.shape(signal_2d[-1])

In [ ]:
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
signal_2d = apply_lct_2d_sep(mx, my, signal_2d)
# 30

In [ ]:
np.shape(signal_2d[-1])

---
## Test the Pieces

In [ ]:
# create initial Gaussian beam
sigmar = 0.15e-3        # initial RMS beam size at center of cavity [m]
propLen=15              # propagation of initial Gaussian [m] (it will back-propagated by propLen to obtain the Gaussian mode at the waist)
pulseE = .001           # total energy of pulse [eV]
mx = 1                  # horizontal mode number (0 for Gaussian)
my = 1                  # vertical mode number (0 for Gaussian)
poltype = 1             # polarization of mode (0=linear horiz, 1=linear vert, 2=linear 45°, 3=linear 135°, 4=circular right, 5=circular left, 6=total)
wavefrontEnergy = 1.55  # wavefront Energy [eV]; 1.55 eV <==> 800 nm wavelength
sampFact = 10           # determines grid size of initial wavefront
wfr0=createGsnSrcSRW(sigmar,propLen,pulseE,poltype,wavefrontEnergy,sampFact,mx,my) # create Gaussian wavefront in SRW
(nx0,ny0)=(wfr0.mesh.nx,wfr0.mesh.ny)
print("Initial wavefront calculated on a grid of %d x %d points" %(nx0,ny0))

In [ ]:
#Resize initial Elec Field?
srwl.ResizeElecField(wfr0, 'c', [0, 1.5, 1, 1.5, 1])
(nx0, ny0) = (wfr0.mesh.nx, wfr0.mesh.ny)
print("New grid size of %d x %d points" %(nx0, ny0))

In [ ]:
?srwl.ResizeElecField

In [ ]:
# create an ABCD rotation matrix
theta = np.pi/4
Aval =  np.cos(theta)
Bval =  np.sin(theta)
Cval = -np.sin(theta)
Dval =  np.cos(theta)

# create a deep copy of the initial Gaussian wavefront
# (because we wish to keep a copy of the original)
wfr1 = deepcopy(wfr0)

#wfrsizef=(wfr1.mesh.nx*wfr1.mesh.ny)

ABCD_BL = createABCDbeamline(Aval,Bval,Cval,Dval) 

start_time = time.time()
srwl.PropagElecField(wfr1, ABCD_BL) #theta = pi/4  
wfr2 = deepcopy(wfr1)
srwl.PropagElecField(wfr2, ABCD_BL) #theta =  pi/2
srwl.PropagElecField(wfr2, ABCD_BL) #theta = 3pi/4
srwl.PropagElecField(wfr2, ABCD_BL) #theta =  pi
srwl.PropagElecField(wfr2, ABCD_BL) #theta = 5pi/4
srwl.PropagElecField(wfr2, ABCD_BL) #theta = 3pi/2
srwl.PropagElecField(wfr2, ABCD_BL) #theta = 7pi/4
srwl.PropagElecField(wfr2, ABCD_BL) #theta = 2pi
print("Propagation time: %s seconds" %(round((time.time() - start_time),5)))

In [ ]:
print(np.shape(wfr2.arEx), np.min(wfr2.arEx), np.max(wfr2.arEx))
print(np.shape(wfr2.arEy), np.min(wfr2.arEy), np.max(wfr2.arEy))

In [ ]:
17006112 / 2916 ** 2

In [ ]:
srwl.CalcIntFromElecField?

https://github.com/ochubar/SRW/blob/master/env/work/srw_python/srwlib.py#L10708

In [ ]:
np.reshape?

In [ ]:
def elec_v_xy(wfr, i_re_im):
    """
    Given the SRW wavefront object wfr, return 
    """
    i_pol  = 6            # total
    i_type = 5 + i_re_im  # i_re_im = 0 (real) or 1 (imag)
    i_dep  = 3            # x&y
    
    xvals = np.linspace(wfr.mesh.xStart, wfr.mesh.xFin, wfr.mesh.nx)
    yvals = np.linspace(wfr.mesh.yStart, wfr.mesh.yFin, wfr.mesh.ny)

    arr = array('f', [0] * wfr.mesh.nx * wfr.mesh.ny)
    srwl.CalcIntFromElecField(arr, wfr, i_pol, i_type, i_dep, wfr.mesh.eStart, 0, 0)
    arr = np.array(arr).reshape((wfr.mesh.nx, wfr.mesh.ny), order = 'C')

    return (xvals, yvals, arr)

In [ ]:
xv, yv, e2d =  elec_v_xy(wfr0, 0)

In [ ]:
e2d.shape

In [ ]:
# plot initial wavefront
wfrplot = wfr0

arIinit = array('f', [0] * wfrplot.mesh.nx * wfrplot.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIinit, wfrplot, 6, 0, 3, wfrplot.mesh.eStart, 0, 0) #extracts intensity
##Reshaping electric field data from flat to 2D array
arIinit2D = np.array(arIinit).reshape((wfrplot.mesh.nx, wfrplot.mesh.ny), order='C')
wfrsizei=np.size(arIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvals0=np.linspace(wfrplot.mesh.xStart,wfrplot.mesh.xFin,wfrplot.mesh.nx)
yvals0=np.linspace(wfrplot.mesh.yStart,wfrplot.mesh.yFin,wfrplot.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

#By setting wfr0a.unitElFldAng=1, default units should now be in mrad(?)
plt.pcolormesh(np.multiply(xvals0,1e6), np.multiply(yvals0,1e6), arIinit2D, cmap=plt.cm.viridis)
plt.colorbar()
zoomfactor=10
xmax = wfrplot.mesh.xFin*1e6
ymax = wfrplot.mesh.yFin*1e6
ax.set_ylim(-ymax/zoomfactor/1.8, ymax/zoomfactor/1.8)
ax.set_xlim(-xmax/zoomfactor/1.8, xmax/zoomfactor/1.8)
ax.set_aspect(1.0)
ax.set_ylabel(r'Vertical Position [$\mu m$]')
ax.set_xlabel(r'Horizontal Position [$\mu m$]')
ax.set_title('Initial Intensity')
# ax.xtick.labelsize
plt.savefig('LCT-0.png')
plt.show()

In [ ]:
# Plot wavefront at pi/4
wfrplot2 = wfr1

arIinit = array('f', [0]*wfrplot2.mesh.nx*wfrplot2.mesh.ny) #"flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIinit, wfrplot2, 6, 0, 3, wfrplot2.mesh.eStart, 0, 0) #extracts intensity
##Reshaping electric field data from flat to 2D array
arIinit2D1 = np.array(arIinit).reshape((wfrplot2.mesh.nx, wfrplot2.mesh.ny), order='C')
wfrsizei=np.size(arIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvals0=np.linspace(wfrplot2.mesh.xStart,wfrplot2.mesh.xFin,wfrplot2.mesh.nx)
yvals0=np.linspace(wfrplot2.mesh.yStart,wfrplot2.mesh.yFin,wfrplot2.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

plt.pcolormesh(np.multiply(xvals0,1e6), np.multiply(yvals0,1e6), arIinit2D1, cmap=plt.cm.viridis)  ##By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
plt.colorbar()
ax.set_ylim(-ymax/zoomfactor,ymax/zoomfactor)
ax.set_xlim(-xmax/zoomfactor,xmax/zoomfactor)
ax.set_aspect(1.0)
ax.set_ylabel(r'Vertical Position [$\mu m$]')
ax.set_xlabel(r'Horizontal Position [$\mu m$]')
ax.set_title('Intensity after pi/4 rotation')
plt.savefig('LCT-1.png')
plt.show()

In [ ]:
# Plot final wavefront
wfrplot2 = wfr2

arIfinal = array('f', [0]*wfrplot2.mesh.nx*wfrplot2.mesh.ny) #"flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIfinal, wfrplot2, 6, 0, 3, wfrplot2.mesh.eStart, 0, 0) #extracts intensity
##Reshaping electric field data from flat to 2D array
arIfinal2D = np.array(arIfinal).reshape((wfrplot2.mesh.nx, wfrplot2.mesh.ny), order='C')
wfrsizei=np.size(arIfinal)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvals0=np.linspace(wfrplot2.mesh.xStart,wfrplot2.mesh.xFin,wfrplot2.mesh.nx)
yvals0=np.linspace(wfrplot2.mesh.yStart,wfrplot2.mesh.yFin,wfrplot2.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

plt.pcolormesh(np.multiply(xvals0,1e6), np.multiply(yvals0,1e6), arIfinal2D, cmap=plt.cm.viridis)  # setting wfr0a.unitElFldAng=1, default units should now be in mrad(?)
plt.colorbar()
ax.set_ylim(-ymax/zoomfactor/1.8, ymax/zoomfactor/1.8)
ax.set_xlim(-xmax/zoomfactor/1.8, xmax/zoomfactor/1.8)
ax.set_ylabel(r'Vertical Position [$\mu m$]')
ax.set_xlabel(r'Horizontal Position [$\mu m$]')
ax.set_aspect(1.0)
ax.set_title('Final Intensity after 2pi rotation')
plt.savefig('LCT-8.png')
plt.show()

In [ ]:
arIDif = arIfinal2D - arIinit2D
difMax = np.max(arIDif)
print('%.10f' % difMax)

In [ ]:
# Plot difference in intensity
wfrplot2 = wfr2

norm = np.max(np.abs(arIinit2D))

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvals0=np.linspace(wfrplot2.mesh.xStart,wfrplot2.mesh.xFin,wfrplot2.mesh.nx)
yvals0=np.linspace(wfrplot2.mesh.yStart,wfrplot2.mesh.yFin,wfrplot2.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

plt.pcolormesh(np.multiply(xvals0,1e6), np.multiply(yvals0,1e6), arIDif/norm, cmap=plt.cm.viridis)  ##By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
plt.colorbar()
ax.set_ylim(-ymax/zoomfactor/1.8, ymax/zoomfactor/1.8)
ax.set_xlim(-xmax/zoomfactor/1.8, xmax/zoomfactor/1.8)
ax.set_ylabel(r'Vertical Position [$\mu m$]')
ax.set_xlabel(r'Horizontal Position [$\mu m$]')
ax.set_aspect(1.0)
ax.set_title('Intensity difference')
plt.savefig('LCT-dif.png')
plt.show()

In [ ]:
# Plot final wavefront
wfrplot2 = wfr2

arIfinal = array('f', [0]*wfrplot2.mesh.nx*wfrplot2.mesh.ny) #"flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIfinal, wfrplot2, 6, 0, 3, wfrplot2.mesh.eStart, 0, 0) #extracts intensity
##Reshaping electric field data from flat to 2D array
arIfinal2D = np.array(arIfinal).reshape((wfrplot2.mesh.nx, wfrplot2.mesh.ny), order='C')
wfrsizei=np.size(arIfinal)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvals0=np.linspace(wfrplot2.mesh.xStart,wfrplot2.mesh.xFin,wfrplot2.mesh.nx)
yvals0=np.linspace(wfrplot2.mesh.yStart,wfrplot2.mesh.yFin,wfrplot2.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

plt.pcolormesh(np.multiply(xvals0,1e6), np.multiply(yvals0,1e6), arIfinal2D, cmap=plt.cm.viridis)  ##By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
plt.colorbar()
ax.set_ylim(-ymax/zoomfactor,ymax/zoomfactor)
ax.set_xlim(-xmax/zoomfactor,xmax/zoomfactor)
ax.set_ylabel(r'Vertical Position [$\mu m$]',**hfontLarge)
ax.set_xlabel(r'Horizontal Position [$\mu m$]',**hfontLarge)
ax.set_title('Final Intensity after 2pi rotation',**hfontLarge)
plt.show()

In [ ]:
arIDif = arIfinal2D - arIinit2D
difMax = np.max(arIDif)
print('%.10f' % difMax)

In [ ]:
# Plot difference in intensity
wfrplot2 = wfr2

norm = np.max(np.abs(arIinit2D))

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvals0=np.linspace(wfrplot2.mesh.xStart,wfrplot2.mesh.xFin,wfrplot2.mesh.nx)
yvals0=np.linspace(wfrplot2.mesh.yStart,wfrplot2.mesh.yFin,wfrplot2.mesh.ny)

fig = plt.figure(figsize=(12,7))
ax = fig.gca()

plt.pcolormesh(np.multiply(xvals0,1e6), np.multiply(yvals0,1e6), arIDif/norm, cmap=plt.cm.viridis)  ##By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
plt.colorbar()
ax.set_ylim(-ymax/zoomfactor,ymax/zoomfactor)
ax.set_xlim(-xmax/zoomfactor,xmax/zoomfactor)
ax.set_ylabel(r'Vertical Position [$\mu m$]',**hfontLarge)
ax.set_xlabel(r'Horizontal Position [$\mu m$]',**hfontLarge)
ax.set_title('Intensity difference',**hfontLarge)
plt.show()